In [22]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [23]:
learning_rate = 0.001
num_steps = 2000
batch_size = 128

num_input = 784
num_classes = 10
dropout = 0.25

In [24]:
#create neural net
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    with tf.variable_scope('ConvNet', reuse=reuse):
        x = x_dict['images']
        
        x = tf.reshape(x, shape=[-1, 28, 28, 1])
        
        conv1 = tf.layers.conv2d(x, 32, 5, activation = tf.nn.relu)
        print("conv1", conv1.shape)
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        print("conv1pool", conv1.shape)
        
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation = tf.nn.relu)
        print("conv2", conv2.shape)
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        print("conv2pool", conv2.shape)
        fc1 = tf.contrib.layers.flatten(conv2)
        
        fc1 = tf.layers.dense(fc1, 1024)
        
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        out = tf.layers.dense(fc1, n_classes)
        
    return out

In [25]:
def model_fn(features, labels, mode):
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    pred_class = tf.argmax(logits_test, axis=1)
    pred_prob = tf.nn.softmax(logits_test)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_class) 
    
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_class)
    
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_class,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [26]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/4g/t6r33qyj7ts58ygnfj7htgq80000gn/T/tmp_tf7n_09', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c2b3ce8d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [27]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Calling model_fn.
conv1 (128, 24, 24, 32)
conv1pool (128, 12, 12, 32)
conv2 (128, 10, 10, 64)
conv2pool (128, 5, 5, 64)
conv1 (128, 24, 24, 32)
conv1pool (128, 12, 12, 32)
conv2 (128, 10, 10, 64)
conv2pool (128, 5, 5, 64)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/4g/t6r33qyj7ts58ygnfj7htgq80000gn/T/tmp_tf7n_09/model.ckpt.
INFO:tensorflow:loss = 2.3283, step = 1
INFO:tensorflow:global_step/sec: 11.7548
INFO:tensorflow:loss = 0.12882046, step = 101 (8.508 sec)
INFO:tensorflow:global_step/sec: 11.6389
INFO:tensorflow:loss = 0.03754104, step = 201 (8.592 sec)
INFO:tensorflow:global_step/sec: 11.4064
INFO:tensorflow:loss = 0.030222854, step = 301 (8.767 sec)
INFO:tensorflow:global_step/sec: 12.4487
INFO:tensorflow:loss = 0.11703863, step = 401 (8.033 sec)
INF